In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import statsmodels.formula.api as sm

from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

import matplotlib.pylab as plt



no display found. Using non-interactive Agg backend


In [2]:
#1a.  
boston_df = pd.read_csv('BostonHousing.csv')

 
boston_df.shape

(506, 14)

In [3]:
boston_df.head()

,CRIME,ZONE,INDUST,CHAR RIV,NIT OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST RATIO,LOW STAT,MVALUE,C MVALUE
0,0.00632,18.0,2.31,N,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,No
1,0.02731,0.0,7.07,N,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,No
2,0.02729,0.0,7.07,N,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,Yes
3,0.03237,0.0,2.18,N,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,Yes
4,0.06905,0.0,2.18,N,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,Yes


In [4]:
boston_df.columns

Index(['CRIME', 'ZONE', 'INDUST', 'CHAR RIV', 'NIT OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST RATIO', 'LOW STAT', 'MVALUE',
       'C MVALUE'],
      dtype='object')

In [5]:
#1b.
boston_df.columns = boston_df.columns.str.replace(' ', '_')
boston_df.columns

Index(['CRIME', 'ZONE', 'INDUST', 'CHAR_RIV', 'NIT_OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'MVALUE',
       'C_MVALUE'],
      dtype='object')

In [6]:
#1c.
boston_df.dtypes

CRIME        float64
ZONE         float64
INDUST       float64
CHAR_RIV      object
NIT_OXIDE    float64
ROOMS        float64
AGE          float64
DISTANCE     float64
RADIAL         int64
TAX            int64
ST_RATIO     float64
LOW_STAT     float64
MVALUE       float64
C_MVALUE      object
dtype: object

In [7]:
#1c.
boston_df.CHAR_RIV = boston_df.CHAR_RIV.astype('category')


print(' ')
print('Category levels and changed variable type:')
print(boston_df.CHAR_RIV.cat.categories) 
print(boston_df.CHAR_RIV.dtype)

 
Category levels and changed variable type:
Index(['N', 'Y'], dtype='object')
category


In [14]:
#1c.
boston_df.C_MVALUE = boston_df.C_MVALUE.astype('category')

a
print(' ')
print('Category levels and changed variable type:')a
print(boston_df.C_MVALUE.cat.categories) 
print(boston_df.C_MVALUE.dtype)

 
Category levels and changed variable type:
Index(['No', 'Yes'], dtype='object')
category


In [8]:
#1c.
boston_df = pd.get_dummies(boston_df, prefix_sep='_', 
                            drop_first=True)
boston_df.columns

Index(['CRIME', 'ZONE', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'AGE', 'DISTANCE',
       'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'MVALUE', 'CHAR_RIV_Y',
       'C_MVALUE_Yes'],
      dtype='object')

In [9]:
#1d.
boston_df.describe()

,CRIME,ZONE,INDUST,NIT_OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST_RATIO,LOW_STAT,MVALUE,CHAR_RIV_Y,C_MVALUE_Yes
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806,0.069170,0.166008
std,8.601545,23.322453,6.860353,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104,0.253994,0.372456
min,0.006320,0.000000,0.460000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000,0.000000,0.000000
25%,0.082045,0.000000,5.190000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000,0.000000,0.000000
50%,0.256510,0.000000,9.690000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000,0.000000,0.000000
75%,3.677083,12.500000,18.100000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000,0.000000,0.000000
max,88.976200,100.000000,27.740000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000,1.000000,1.000000


In [10]:
#1d. 
boston_df.isnull().sum()

CRIME           0
ZONE            0
INDUST          0
NIT_OXIDE       0
ROOMS           0
AGE             0
DISTANCE        0
RADIAL          0
TAX             0
ST_RATIO        0
LOW_STAT        0
MVALUE          0
CHAR_RIV_Y      0
C_MVALUE_Yes    0
dtype: int64

In [11]:
#2a. Identify predictors and outcome of the regression model.
predictors = ['CRIME', 'ZONE', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'AGE', 'DISTANCE',
       'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT','CHAR_RIV_Y','C_MVALUE_Yes']
outcome = 'MVALUE'


X = boston_df[predictors]
y = boston_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=1)


boston_lm = LinearRegression()
boston_lm.fit(train_X, train_y)


print('Regression Model for boston housing Training Set')
print()
print('Intercept: ', np.round(boston_lm.intercept_, 2)) 
print(pd.DataFrame({'Predictor': X.columns, 'Coefficient': np.round(boston_lm.coef_, 2)}))


Regression Model for boston housing Training Set

Intercept:  46.41
       Predictor  Coefficient
0          CRIME        -0.13
1           ZONE        -0.01
2         INDUST         0.11
3      NIT_OXIDE       -17.12
4          ROOMS         0.64
5            AGE        -0.01
6       DISTANCE        -0.70
7         RADIAL         0.19
8            TAX        -0.01
9       ST_RATIO        -0.60
10      LOW_STAT        -0.47
11    CHAR_RIV_Y         2.17
12  C_MVALUE_Yes        11.66


In [12]:
#2b.Use predict() to score (make) predictions for validation set.
boston_lm_pred = boston_lm.predict(valid_X)

print('Actual, Prediction, and Residual Prices for Validation Set')
result = round(pd.DataFrame({'Actual': valid_y,'Predicted': boston_lm_pred, 
                       'Residual': valid_y - boston_lm_pred}), 2)
print(result.head(10))

Actual, Prediction, and Residual Prices for Validation Set
     Actual  Predicted  Residual
307    28.2      25.24      2.96
343    23.9      22.44      1.46
47     16.6      18.11     -1.51
67     22.0      22.19     -0.19
362    20.8      19.07      1.73
132    23.0      19.54      3.46
292    27.9      25.10      2.80
31     14.5      18.10     -3.60
218    21.5      22.55     -1.05
90     22.6      23.58     -0.98


In [13]:
#2b. 
pred_y = boston_lm.predict(train_X)

r2 = round(r2_score(train_y, pred_y),3)
adj_r2 = round(adjusted_r2_score(train_y, pred_y, boston_lm),3)
aic = round(AIC_score(train_y, pred_y, boston_lm),2)
bic = round(BIC_score(train_y, pred_y, boston_lm),2)

print('Prediction Performance Measures for Training Set')
print('r2 : ', r2)
print('Adjusted r2 : ', adj_r2)
print('AIC : ', aic)
print('BIC : ', bic)
print() 

r2 = round(r2_score(valid_y, boston_lm_pred),3)
adj_r2 = round(adjusted_r2_score(valid_y, boston_lm_pred, boston_lm),3)
aic = round(AIC_score(valid_y, boston_lm_pred, boston_lm),2)
bic = round(BIC_score(valid_y, boston_lm_pred, boston_lm),2)

print('Prediction Performance Measures for Validation Set')
print('r2 : ', r2)
print('adjusted r2 : ', adj_r2)
print('AIC : ', aic)
print('BIC : ', bic)

Prediction Performance Measures for Training Set
r2 :  0.836
Adjusted r2 :  0.831
AIC :  2220.32
BIC :  2280.34

Prediction Performance Measures for Validation Set
r2 :  0.851
adjusted r2 :  0.829
AIC :  593.83
BIC :  633.2


In [14]:
#2c. Display common accuracy measures for training set.
print('Accuracy Measures for Training Set - All Variables')
regressionSummary(train_y, pred_y)
print()

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - All Variables')
regressionSummary(valid_y, boston_lm_pred)

Accuracy Measures for Training Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 3.6395
            Mean Absolute Error (MAE) : 2.6454
          Mean Percentage Error (MPE) : -2.6958
Mean Absolute Percentage Error (MAPE) : 12.9926

Accuracy Measures for Validation Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.2023
       Root Mean Squared Error (RMSE) : 3.8378
            Mean Absolute Error (MAE) : 2.8230
          Mean Percentage Error (MPE) : -4.5533
Mean Absolute Percentage Error (MAPE) : 14.6529


In [15]:
#3a. 
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model_be, best_variables_be = backward_elimination(train_X.columns, 
                        train_model, score_model, verbose=True)

print()
print('Best Variables from Backward Elimination Algorithm')
print(best_variables_be)

Variables: CRIME, ZONE, INDUST, NIT_OXIDE, ROOMS, AGE, DISTANCE, RADIAL, TAX, ST_RATIO, LOW_STAT, CHAR_RIV_Y, C_MVALUE_Yes
Start: score=2220.32
Step: score=2218.56, remove AGE
Step: score=2216.85, remove ZONE
Step: score=2216.85, remove None

Best Variables from Backward Elimination Algorithm
['CRIME', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y', 'C_MVALUE_Yes']


In [16]:
#3a.
predictors_be = ['CRIME', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y', 'C_MVALUE_Yes']
outcome = 'MVALUE'
 
X = boston_df[predictors_be]
y = boston_df[outcome]
train_X_be, valid_X_be, train_y_be, valid_y_be = \
          train_test_split(X, y, test_size=0.2, random_state=1)

boston_be = LinearRegression()
boston_be.fit(train_X_be, train_y_be)

print('Regression Model for Training Set Using Backward Elimination')
print()
print('Intercept ', np.round(boston_be.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_be.coef_, 2)}))

Regression Model for Training Set Using Backward Elimination

Intercept  46.55
       Predictor  Coefficient
0          CRIME        -0.13
1         INDUST         0.11
2      NIT_OXIDE       -17.47
3          ROOMS         0.61
4       DISTANCE        -0.73
5         RADIAL         0.20
6            TAX        -0.01
7       ST_RATIO        -0.59
8       LOW_STAT        -0.48
9     CHAR_RIV_Y         2.14
10  C_MVALUE_Yes        11.52


In [17]:
#3a.
boston_be_pred = boston_be.predict(valid_X_be)

result = round(pd.DataFrame({'Actual': valid_y_be,'Predicted': boston_be_pred, 
                       'Residual': valid_y_be - boston_be_pred}), 2)
print()
print('Predictions for Validation Set Using Backward Elimination')
print(result.head(10))

print()
print('Accuracy Measures for Validation Set Using Backward Elimination')
regressionSummary(valid_y_be, boston_be_pred)


Predictions for Validation Set Using Backward Elimination
     Actual  Predicted  Residual
307    28.2      25.57      2.63
343    23.9      22.70      1.20
47     16.6      18.11     -1.51
67     22.0      21.98      0.02
362    20.8      19.19      1.61
132    23.0      19.68      3.32
292    27.9      25.52      2.38
31     14.5      18.25     -3.75
218    21.5      22.62     -1.12
90     22.6      23.57     -0.97

Accuracy Measures for Validation Set Using Backward Elimination

Regression statistics

                      Mean Error (ME) : 0.1904
       Root Mean Squared Error (RMSE) : 3.8356
            Mean Absolute Error (MAE) : 2.8137
          Mean Percentage Error (MPE) : -4.5767
Mean Absolute Percentage Error (MAPE) : 14.5939


In [18]:
#3b.
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model_fs, best_variables_fs = forward_selection(train_X.columns, 
                    train_model, score_model, verbose=True)

print()
print('Best Variables from Forward Selection Algorithm')
print(best_variables_fs)

Variables: CRIME, ZONE, INDUST, NIT_OXIDE, ROOMS, AGE, DISTANCE, RADIAL, TAX, ST_RATIO, LOW_STAT, CHAR_RIV_Y, C_MVALUE_Yes
Start: score=2924.77, constant
Step: score=2542.89, add C_MVALUE_Yes
Step: score=2295.94, add LOW_STAT
Step: score=2276.57, add CRIME
Step: score=2265.27, add CHAR_RIV_Y
Step: score=2255.86, add ST_RATIO
Step: score=2247.88, add DISTANCE
Step: score=2227.34, add NIT_OXIDE
Step: score=2220.35, add RADIAL
Step: score=2219.47, add TAX
Step: score=2217.31, add INDUST
Step: score=2216.85, add ROOMS
Step: score=2216.85, add None

Best Variables from Forward Selection Algorithm
['C_MVALUE_Yes', 'LOW_STAT', 'CRIME', 'CHAR_RIV_Y', 'ST_RATIO', 'DISTANCE', 'NIT_OXIDE', 'RADIAL', 'TAX', 'INDUST', 'ROOMS']


In [19]:
#3b.
predictors_fs = ['C_MVALUE_Yes', 'LOW_STAT', 'CRIME', 'CHAR_RIV_Y', 'ST_RATIO', 'DISTANCE', 'NIT_OXIDE', 'RADIAL', 'TAX', 'INDUST', 'ROOMS']
outcome = 'MVALUE'
 
X = boston_df[predictors_fs]
y = boston_df[outcome]
train_X_fs, valid_X_fs, train_y_fs, valid_y_fs = \
          train_test_split(X, y, test_size=0.2, random_state=1)
boston_fs = LinearRegression()
boston_fs.fit(train_X_fs, train_y_fs)

print('Regression Model for Training Set Using Forward Selection')
print()
print('Intercept ', np.round(boston_fs.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_fs.coef_, 2)}))

Regression Model for Training Set Using Forward Selection

Intercept  46.55
       Predictor  Coefficient
0   C_MVALUE_Yes        11.52
1       LOW_STAT        -0.48
2          CRIME        -0.13
3     CHAR_RIV_Y         2.14
4       ST_RATIO        -0.59
5       DISTANCE        -0.73
6      NIT_OXIDE       -17.47
7         RADIAL         0.20
8            TAX        -0.01
9         INDUST         0.11
10         ROOMS         0.61


In [20]:
#3b.
boston_fs_pred = boston_fs.predict(valid_X_fs)

result = round(pd.DataFrame({'Actual': valid_y_fs,'Predicted': boston_fs_pred, 
                       'Residual': valid_y_fs - boston_fs_pred}), 2)
print()
print('Predictions for Validation Set Using Forward Selection')
print(result.head(10))
print()
print('Accuracy Measures for Validation Set Using Forward Selection')
regressionSummary(valid_y_fs, boston_fs_pred)


Predictions for Validation Set Using Forward Selection
     Actual  Predicted  Residual
307    28.2      25.57      2.63
343    23.9      22.70      1.20
47     16.6      18.11     -1.51
67     22.0      21.98      0.02
362    20.8      19.19      1.61
132    23.0      19.68      3.32
292    27.9      25.52      2.38
31     14.5      18.25     -3.75
218    21.5      22.62     -1.12
90     22.6      23.57     -0.97

Accuracy Measures for Validation Set Using Forward Selection

Regression statistics

                      Mean Error (ME) : 0.1904
       Root Mean Squared Error (RMSE) : 3.8356
            Mean Absolute Error (MAE) : 2.8137
          Mean Percentage Error (MPE) : -4.5767
Mean Absolute Percentage Error (MAPE) : 14.5939


In [21]:
#3c.
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)


best_model_st, best_variables_st = stepwise_selection(train_X.columns, 
                    train_model, score_model, verbose=True)
print()
print('Best Variables from Stepwise Selection Algorithm')
print(best_variables_st)

Variables: CRIME, ZONE, INDUST, NIT_OXIDE, ROOMS, AGE, DISTANCE, RADIAL, TAX, ST_RATIO, LOW_STAT, CHAR_RIV_Y, C_MVALUE_Yes
Start: score=2924.77, constant
Step: score=2542.89, add C_MVALUE_Yes
Step: score=2295.94, add LOW_STAT
Step: score=2276.57, add CRIME
Step: score=2265.27, add CHAR_RIV_Y
Step: score=2255.86, add ST_RATIO
Step: score=2247.88, add DISTANCE
Step: score=2227.34, add NIT_OXIDE
Step: score=2220.35, add RADIAL
Step: score=2219.47, add TAX
Step: score=2217.31, add INDUST
Step: score=2216.85, add ROOMS
Step: score=2216.85, unchanged None

Best Variables from Stepwise Selection Algorithm
['C_MVALUE_Yes', 'LOW_STAT', 'CRIME', 'CHAR_RIV_Y', 'ST_RATIO', 'DISTANCE', 'NIT_OXIDE', 'RADIAL', 'TAX', 'INDUST', 'ROOMS']


In [22]:
#3c.
predictors_st = ['C_MVALUE_Yes', 'LOW_STAT', 'CRIME', 'CHAR_RIV_Y', 'ST_RATIO', 'DISTANCE', 'NIT_OXIDE', 'RADIAL', 'TAX', 'INDUST', 'ROOMS']
outcome = 'MVALUE'

X = boston_df[predictors_st]
y = boston_df[outcome]
train_X_st, valid_X_st, train_y_st, valid_y_st = \
          train_test_split(X, y, test_size=0.2, random_state=1)

boston_st = LinearRegression()
boston_st.fit(train_X_st, train_y_st)

print('Regression Model for Training Set Using Stewise Selection')
print()
print('Intercept ', np.round(boston_st.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_st.coef_, 2)}))

Regression Model for Training Set Using Stewise Selection

Intercept  46.55
       Predictor  Coefficient
0   C_MVALUE_Yes        11.52
1       LOW_STAT        -0.48
2          CRIME        -0.13
3     CHAR_RIV_Y         2.14
4       ST_RATIO        -0.59
5       DISTANCE        -0.73
6      NIT_OXIDE       -17.47
7         RADIAL         0.20
8            TAX        -0.01
9         INDUST         0.11
10         ROOMS         0.61


In [23]:
#3c.
boston_st_pred = boston_st.predict(valid_X_st)


result = round(pd.DataFrame({'Actual': valid_y_st,'Predicted': boston_st_pred, 
                       'Residual': valid_y_st - boston_st_pred}), 2)
print()
print('Predictions for Validation Set Using Stepwise Selection')
print(result.head(10))

print()
print('Accuracy Measures for Validation Set Using Stepwise Selection')
regressionSummary(valid_y_st, boston_st_pred)


Predictions for Validation Set Using Stepwise Selection
     Actual  Predicted  Residual
307    28.2      25.57      2.63
343    23.9      22.70      1.20
47     16.6      18.11     -1.51
67     22.0      21.98      0.02
362    20.8      19.19      1.61
132    23.0      19.68      3.32
292    27.9      25.52      2.38
31     14.5      18.25     -3.75
218    21.5      22.62     -1.12
90     22.6      23.57     -0.97

Accuracy Measures for Validation Set Using Stepwise Selection

Regression statistics

                      Mean Error (ME) : 0.1904
       Root Mean Squared Error (RMSE) : 3.8356
            Mean Absolute Error (MAE) : 2.8137
          Mean Percentage Error (MPE) : -4.5767
Mean Absolute Percentage Error (MAPE) : 14.5939
